In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from sklearn.preprocessing import LabelEncoder

model_rdf = joblib.load('../ml_data/model_randomforest.plk')
model_svm = joblib.load('../ml_data/model_svm.plk')

rp_encoder = LabelEncoder()
rp_encoder.classes_ = np.load('./data/classes.npy', allow_pickle=True)

bssid_encoder = LabelEncoder()
bssid_encoder.classes_ = np.load('./data/features.npy', allow_pickle=True)

array(['aisle', 'bedroom', 'kitchen', 'livingroom', 'myroom', 'room2',
       'room3', 'studyroom'], dtype=object)

In [1]:
sys.path.append('../collector/')
from collections import defaultdict



scan_wifi = get_wifis()

wifi_dict = defaultdict.fromkeys(wifi_df.columns, 0)
for wifi in scan_wifi:
    if wifi['bssid'] in wifi_dict:
        wifi_dict[wifi['bssid']] = int(wifi['signal'][:-1])

wifi_df = wifi_df.append(pd.DataFrame.from_dict([wifi_dict]))

result_rdf = model_rdf.predict(wifi_df.tail(1))
result_svm = model_svm.predict(wifi_df.tail(1))
proba_rdf = model_rdf.predict_proba(wifi_df.tail(1))
proba_svm = model_svm.predict_proba(wifi_df.tail(1))

print(f'RDF : {rp_name[result_rdf[0]]}')
for i in range(rp_cnt):
    print(f'[{rp_name[i]}] : {proba_rdf[0][i]:.2f}')

print('')

print(f'SVM : {rp_name[result_svm[0]]}')
for i in range(rp_cnt):
    print(f'[{rp_name[i]}] : {proba_svm[0][i]:.2f}')
    
print('\n')


NameError: name 'get_wifis' is not defined